In [4]:
# Plotting Packages
import matplotlib.pyplot as plt
import seaborn as sbn

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150
savefig_options = dict(format="png", dpi=150, bbox_inches="tight")

# Computation packages
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

In [2]:
# Read the text file into a DataFrame without header row and with specific column names
df = pd.read_csv('./data/lamhong2imu.txt', sep=' ', header=None, names=['timestamp', 'Gx', 'Gy', 'Gz', 'Ax', 'Ay', 'Az'])

# Display the DataFrame
print(df)


        timestamp        Gx        Gy        Gz        Ax        Ay         Az
0      437773.020  0.030526 -0.032188 -0.031856 -1.457032  3.898439  -9.480474
1      437773.040 -0.069907  0.023014  0.069775  3.433596 -0.416992 -10.301763
2      437773.060  0.012434 -0.005782  0.005454 -2.471681  2.782228 -10.221685
3      437773.079  0.078553 -0.023342 -0.012033  1.416016 -0.551758  -9.405278
4      437773.099 -0.085605  0.011773  0.013435  1.402345  0.773438 -10.289068
...           ...       ...       ...       ...       ...       ...        ...
45657  438817.059 -0.063920  0.213582  0.013904  3.509767 -2.132814 -10.810552
45658  438817.078  0.091387 -0.284413 -0.025541 -4.244143  1.204102  -9.227544
45659  438817.098 -0.285815  0.281026  0.082677  3.607424  4.361330  -9.300786
45660  438817.118  0.334630 -0.263794 -0.149921 -4.773440 -5.686526 -11.771490
45661  438817.138 -0.280493  0.153317  0.146998  6.512699  9.839849  -8.175785

[45662 rows x 7 columns]


In [5]:
def compute_euclidean_distance_matrix(x, y) -> np.array:
    """Calculate distance matrix
    This method calcualtes the pairwise Euclidean distance between two sequences.
    The sequences can have different lengths.
    """
    dist = np.zeros((len(y), len(x)))
    for i in range(len(y)):
        for j in range(len(x)):
            dist[i,j] = (x[j]-y[i])**2
    return dist

In [6]:
def compute_accumulated_cost_matrix(x, y) -> np.array:
    """Compute accumulated cost matrix for warp path using Euclidean distance
    """
    distances = compute_euclidean_distance_matrix(x, y)

    # Initialization
    cost = np.zeros((len(y), len(x)))
    cost[0,0] = distances[0,0]
    
    for i in range(1, len(y)):
        cost[i, 0] = distances[i, 0] + cost[i-1, 0]  
        
    for j in range(1, len(x)):
        cost[0, j] = distances[0, j] + cost[0, j-1]  

    # Accumulated warp path cost
    for i in range(1, len(y)):
        for j in range(1, len(x)):
            cost[i, j] = min(
                cost[i-1, j],    # insertion
                cost[i, j-1],    # deletion
                cost[i-1, j-1]   # match
            ) + distances[i, j] 
            
    return cost
